In [26]:
import pandas as pd

# Load Train data
train_df = pd.read_csv('../data/TrainingDataPD25/TrainActivities.csv')
train_df.head()

,ID,Activity Type ID,Activity Type,Started,Finished,Updated,Subject
0,1130251,2806,1 (FACING camera) Sit and stand,2024/09/02 6:16,2024/09/02 6:16,2024/09/02 6:16,U22
1,1130254,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024/09/02 6:17,2024/09/02 6:17,2024/09/02 6:17,U22
2,1130257,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024/09/02 6:18,2024/09/02 6:18,2024/09/02 6:18,U22
3,1130261,2806,1 (FACING camera) Sit and stand,2024/09/02 6:20,2024/09/02 6:20,2024/09/02 6:20,U22
4,1130292,2806,1 (FACING camera) Sit and stand,2024/09/02 6:42,2024/09/02 6:42,2024/09/02 6:42,U2


In [27]:
"""
Reminder:
- Skip this if the dataframe are converted
- Don't use tz_localize(UTC) if the column does not have timezone information
E.g: if the datetime are already in JST, but doesn't cotain UTC information, run this code will cause the datatime set to UTC+0:00 then shift to UTC+9:00.
Which makes the time cannot be matched later on.
"""
# convert Started, Finished and Updated columns to datetime format
# train_df['Started'] = pd.to_datetime(train_df['Started'])
# train_df['Finished'] = pd.to_datetime(train_df['Finished'])
# train_df['Updated'] = pd.to_datetime(train_df['Updated'])

# Convert the timezone to JST
# train_df['Started'] = train_df['Started'].dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')
# train_df['Finished'] = train_df['Finished'].dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')
# train_df['Updated'] = train_df['Updated'].dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')

"\nReminder:\n- Skip this if the dataframe are converted\n- Don't use tz_localize(UTC) if the column does not have timezone information\nE.g: if the datetime are already in JST, but doesn't cotain UTC information, run this code will cause the datatime set to UTC+0:00 then shift to UTC+9:00.\nWhich makes the time cannot be matched later on.\n"

In [28]:
# install my own library for preprocessing
%pip install -e ../lib/machine_learning_nyan/

Obtaining file:///Users/tptn/MachineLearning/tremor_challenge2/lib/machine_learning_nyan
  Preparing metadata (setup.py) ... done
  Attempting uninstall: preprocessing_nyan
    Found existing installation: preprocessing_nyan 0.1.0
    Uninstalling preprocessing_nyan-0.1.0:
      Successfully uninstalled preprocessing_nyan-0.1.0
  DEPRECATION: Legacy editable install of preprocessing_nyan==0.1.0 from file:///Users/tptn/MachineLearning/tremor_challenge2/lib/machine_learning_nyan (setup.py develop) is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for preprocessing_nyan

[notice] A new release of pip is available: 2

In [29]:
from preprocessing_nyan.time_conversion import convert_to_jst

cols = ['Started', 'Finished', 'Updated']
convert_to_jst(train_df, columns=cols)

,ID,Activity Type ID,Activity Type,Started,Finished,Updated,Subject
0,1130251,2806,1 (FACING camera) Sit and stand,2024-09-02 15:16:00+09:00,2024-09-02 15:16:00+09:00,2024-09-02 15:16:00+09:00,U22
1,1130254,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024-09-02 15:17:00+09:00,2024-09-02 15:17:00+09:00,2024-09-02 15:17:00+09:00,U22
2,1130257,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024-09-02 15:18:00+09:00,2024-09-02 15:18:00+09:00,2024-09-02 15:18:00+09:00,U22
3,1130261,2806,1 (FACING camera) Sit and stand,2024-09-02 15:20:00+09:00,2024-09-02 15:20:00+09:00,2024-09-02 15:20:00+09:00,U22
4,1130292,2806,1 (FACING camera) Sit and stand,2024-09-02 15:42:00+09:00,2024-09-02 15:42:00+09:00,2024-09-02 15:42:00+09:00,U2
...,...,...,...,...,...,...,...
337,1164181,2815,"10 Slow walk (SHAKING hands/body, tiny step, h...",NaT,NaT,2024-09-11 13:55:00+09:00,U7
338,1164187,2806,1 (FACING camera) Sit and stand,2024-09-11 13:58:00+09:00,2024-09-11 13:58:00+09:00,2024-09-11 13:58:00+09:00,U7
339,1164217,2815,"10 Slow walk (SHAKING hands/body, tiny step, h...",2024-09-11 14:09:00+09:00,2024-09-11 14:10:00+09:00,2024-09-11 14:10:00+09:00,U7
340,1164218,2815,"10 Slow walk (SHAKING hands/body, tiny step, h...",2024-09-11 14:10:00+09:00,2024-09-11 14:10:00+09:00,2024-09-11 14:10:00+09:00,U7


In [39]:
# because start and finish don't contain seconds, so we need to add them inorder to create interval
# train_df['start_at'] = train_df['Started'].dt.floor('s')
# train_df['finish_at'] = train_df['Finished'].dt.floor('min') + pd.Timedelta(minutes=1) - pd.Timedelta(microseconds=1)
train_df_cp = train_df.copy()
train_df_cp['start_at'] = train_df_cp['Started'].dt.floor('us')+ pd.Timedelta(microseconds=1)- pd.Timedelta(microseconds=1)
display(train_df_cp[['Started', 'start_at']].head())

,Started,start_at
0,2024-09-02 15:16:00+09:00,2024-09-02 15:16:00+09:00
1,2024-09-02 15:17:00+09:00,2024-09-02 15:17:00+09:00
2,2024-09-02 15:18:00+09:00,2024-09-02 15:18:00+09:00
3,2024-09-02 15:20:00+09:00,2024-09-02 15:20:00+09:00
4,2024-09-02 15:42:00+09:00,2024-09-02 15:42:00+09:00


In [21]:
train_df.head()

,ID,Activity Type ID,Activity Type,Started,Finished,Updated,Subject,start_at,finish_at
0,1130251,2806,1 (FACING camera) Sit and stand,2024-09-02 15:16:00+09:00,2024-09-02 15:16:00+09:00,2024-09-02 15:16:00+09:00,U22,2024-09-02 15:16:00+09:00,2024-09-02 15:16:59.999999+09:00
1,1130254,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024-09-02 15:17:00+09:00,2024-09-02 15:17:00+09:00,2024-09-02 15:17:00+09:00,U22,2024-09-02 15:17:00+09:00,2024-09-02 15:17:59.999999+09:00
2,1130257,2807,2 (FACING camera) both hands SHAKING (sitting ...,2024-09-02 15:18:00+09:00,2024-09-02 15:18:00+09:00,2024-09-02 15:18:00+09:00,U22,2024-09-02 15:18:00+09:00,2024-09-02 15:18:59.999999+09:00
3,1130261,2806,1 (FACING camera) Sit and stand,2024-09-02 15:20:00+09:00,2024-09-02 15:20:00+09:00,2024-09-02 15:20:00+09:00,U22,2024-09-02 15:20:00+09:00,2024-09-02 15:20:59.999999+09:00
4,1130292,2806,1 (FACING camera) Sit and stand,2024-09-02 15:42:00+09:00,2024-09-02 15:42:00+09:00,2024-09-02 15:42:00+09:00,U2,2024-09-02 15:42:00+09:00,2024-09-02 15:42:59.999999+09:00


In [149]:
train_df = train_df.sort_values(by=['Subject', 'start_at'])
train_df.head()

,ID,Activity Type ID,Activity Type,Started,Finished,Updated,Subject,start_at,finish_at
7,1130306,2806,1 (FACING camera) Sit and stand,2024-09-02 15:44:00+09:00,2024-09-02 15:44:00+09:00,2024-09-02 15:44:00+09:00,U1,2024-09-02 15:44:00+09:00,2024-09-02 15:44:59.999000+09:00
118,1137246,2806,1 (FACING camera) Sit and stand,2024-09-04 06:21:00+09:00,2024-09-04 06:21:00+09:00,2024-09-04 06:42:00+09:00,U1,2024-09-04 06:21:00+09:00,2024-09-04 06:21:59.999000+09:00
119,1137247,2806,1 (FACING camera) Sit and stand,2024-09-04 06:22:00+09:00,2024-09-04 06:22:00+09:00,2024-09-04 06:42:00+09:00,U1,2024-09-04 06:22:00+09:00,2024-09-04 06:22:59.999000+09:00
120,1137248,2806,1 (FACING camera) Sit and stand,2024-09-04 06:22:00+09:00,2024-09-04 06:22:00+09:00,2024-09-04 06:42:00+09:00,U1,2024-09-04 06:22:00+09:00,2024-09-04 06:22:59.999000+09:00
146,1137278,2806,1 (FACING camera) Sit and stand,2024-09-04 06:22:00+09:00,2024-09-04 06:22:00+09:00,2024-09-04 06:38:00+09:00,U1,2024-09-04 06:22:00+09:00,2024-09-04 06:22:59.999000+09:00


In [151]:
train_df.isna().sum()
# there are na values in started and finished columns
train_df.dropna(inplace=True)
train_df.drop_duplicates(keep='last', inplace=True)
train_df.reset_index(drop=True, inplace=True) # drop=true removes the old index column
train_df.isna().sum()
print(train_df.shape)
train_df.head()
act_df = train_df.drop(columns=['Started', 'Finished'])
act_df.head()

(275, 9)


,ID,Activity Type ID,Activity Type,Updated,Subject,start_at,finish_at
0,1130306,2806,1 (FACING camera) Sit and stand,2024-09-02 15:44:00+09:00,U1,2024-09-02 15:44:00+09:00,2024-09-02 15:44:59.999000+09:00
1,1137246,2806,1 (FACING camera) Sit and stand,2024-09-04 06:42:00+09:00,U1,2024-09-04 06:21:00+09:00,2024-09-04 06:21:59.999000+09:00
2,1137247,2806,1 (FACING camera) Sit and stand,2024-09-04 06:42:00+09:00,U1,2024-09-04 06:22:00+09:00,2024-09-04 06:22:59.999000+09:00
3,1137248,2806,1 (FACING camera) Sit and stand,2024-09-04 06:42:00+09:00,U1,2024-09-04 06:22:00+09:00,2024-09-04 06:22:59.999000+09:00
4,1137278,2806,1 (FACING camera) Sit and stand,2024-09-04 06:38:00+09:00,U1,2024-09-04 06:22:00+09:00,2024-09-04 06:22:59.999000+09:00


In [ ]:
# combine all dataframes of each user 
import os
import glob
import pytz

"""
The data is taken in JST timezone
"""
base_dir = '/Users/tptn/MachineLearning/tremor_challenge2/data/TrainingDataPD25/users_timeXYZ/users'

headers = ['id', 'datetime', 'x', 'y', 'z']
df_list = [] # to store all csv files
for id in os.listdir(base_dir):
    path = os.path.join(base_dir, id)

    if os.path.isdir(path):
        files = glob.glob(os.path.join(path, '*.csv'))

        for f in files:
            df = pd.read_csv(f, header=None, names=headers) # load csv
            df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%dT%H:%M:%S.%f%z') # conver to datetime
            tz = df['datetime'].dt.tz # get the timezone
            if tz is None:
                df['datetime'] = df['datetime'].dt.tz_localize('Asia/Tokyo') # localize to Asia/Tokyo if timezone is not set
            elif tz != pytz.timezone('Asia/Tokyo'): # convert timezone if it's not in JST
                df['datetime'] = df['datetime'].dt.tz_convert('Asia/Tokyo')
            
            df_list.append(df)
users_data = pd.concat(df_list, ignore_index=True)

In [94]:
users_data.shape
users_data.isna().sum()
# there are no missing values in the data
users_data.reset_index(drop=True, inplace=True)

In [123]:
users_data.sort_values(by=['id', 'datetime'], inplace=True)
users_data.head()

,id,datetime,x,y,z
0,38,2024-09-09 07:31:01.508000+09:00,1.879,8.423,4.108
1,38,2024-09-09 07:31:01.604000+09:00,1.638,8.548,4.405
2,38,2024-09-09 07:31:01.608000+09:00,1.010,8.097,4.594
3,38,2024-09-09 07:31:01.699000+09:00,1.556,8.356,4.329
4,38,2024-09-09 07:31:01.703000+09:00,1.108,8.018,4.913


In [177]:
train_df.head()
act_df.head()
list(act_df.index)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [184]:
# Split accelerometer data per 1 sample (=Activity at one time)
"""
Make Segment.
A segment is a group of continuous sensor data for which feature calculation (statistic calculation) is performed in human activity recognition using sensor data.
"""
from datetime import timedelta

seg_label_list = [] # segment's (1 sample's) label list (label mean "activity_type_id", "user_id" etc.).
seg_list = [] # segment's (1 sample's) accelerometer data list.

count = 0
id_list = []
for i in list(act_df.index):
    started_at = act_df.iloc[i]['start_at']
    finished_at = act_df.iloc[i]['finish_at']
    seq = users_data[(users_data['datetime'] >= started_at) & (users_data['datetime'] <= finished_at)]
    if len(seq) != 0:
        count+=1
        print(count)
        print(i)
        print(seq.shape)
        print(started_at, finished_at)
        id_list.append(i)

print(id_list)
print(len(id_list))

1
119
(375, 5)
2024-09-06 03:17:00+09:00 2024-09-06 03:17:59.999000+09:00
2
120
(375, 5)
2024-09-06 06:25:00+09:00 2024-09-06 06:25:59.999000+09:00
3
121
(375, 5)
2024-09-06 06:25:00+09:00 2024-09-06 06:25:59.999000+09:00
4
122
(375, 5)
2024-09-06 06:25:00+09:00 2024-09-06 06:25:59.999000+09:00
5
123
(375, 5)
2024-09-06 06:25:00+09:00 2024-09-06 06:25:59.999000+09:00
6
124
(750, 5)
2024-09-06 06:25:00+09:00 2024-09-06 06:26:59.999000+09:00
7
125
(375, 5)
2024-09-06 06:25:00+09:00 2024-09-06 06:25:59.999000+09:00
8
126
(750, 5)
2024-09-06 06:25:00+09:00 2024-09-06 06:26:59.999000+09:00
9
127
(375, 5)
2024-09-06 06:26:00+09:00 2024-09-06 06:26:59.999000+09:00
10
128
(375, 5)
2024-09-06 06:26:00+09:00 2024-09-06 06:26:59.999000+09:00
11
129
(375, 5)
2024-09-06 06:26:00+09:00 2024-09-06 06:26:59.999000+09:00
12
130
(375, 5)
2024-09-06 06:26:00+09:00 2024-09-06 06:26:59.999000+09:00
13
131
(375, 5)
2024-09-06 06:26:00+09:00 2024-09-06 06:26:59.999000+09:00
14
132
(750, 5)
2024-09-06 06:26:0

In [159]:
len(seg_label_list)
len(seg_list)
len(act_df)

275

# Feature Extraction

In [10]:
# Extract start and end date of the whole training dataset
date_start = train_df['Started'].min()
date_finish = train_df['Finished'].max()
print(date_start, date_finish)

2024-09-02 15:16:00+09:00 2024-09-11 14:11:00+09:00


From the output above it looks like that the training dataset is conducted only 9 days from 2024-09-02 to 2024-09-11.

In [11]:
"""
Create hourly data frames from start date to end date
Weekday will be used as feature
"""
import datetime
time = date_start
time_list = []
while time <= date_finish:
    for hour in range (0, 24):
        year_month_date_hour = str(time.date()) + '-' + str(hour).zfill(2)
        time_list.append([year_month_date_hour, hour, time, time.weekday()])
    time = time + datetime.timedelta(days=1)

cols = ['year-month-date', 'hour', 'start', 'weekday']
df_y = pd.DataFrame(time_list, columns=cols)
del time_list, time

In [12]:
df_y.head()

,year-month-date,hour,start,weekday
0,2024-09-02-00,0,2024-09-02 15:16:00+09:00,0
1,2024-09-02-01,1,2024-09-02 15:16:00+09:00,0
2,2024-09-02-02,2,2024-09-02 15:16:00+09:00,0
3,2024-09-02-03,3,2024-09-02 15:16:00+09:00,0
4,2024-09-02-04,4,2024-09-02 15:16:00+09:00,0


In [ ]:
# add year-month-date-hour column to train_df
train_df['year-month-date-hour'] = train_df['Started'].dt.date.astype(str) + '-' + train_df['Started'].dt.hour.astype(str)

In [ ]:
train_cp = train_df.copy()
feat = train_cp.groupby([])

KeyError: "['id'] not in index"